# Experiments

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.prep_data import load_data, split_data
from utils.train import train
from utils.dataset import WindFarmDataset
from GCGRU.GRU import GRU
import torch
from torch.utils.data import DataLoader
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
data = load_data(columns=["TurbID", "P_norm", "datetime"])
nan_mask = data.isna().to_numpy()
data = data.fillna(0)
train_data, val_data, test_data = split_data(data, splits=[0.7, 0.2, 0.1])

### RNN-based models

In [3]:
def format_sliding_window(data: np.ndarray, t: int = 6 * 24):
    X_window, y_window = [], []
    for i in range(data.shape[0] - t - 1):
        a = data[i:(i+t)]
        X_window.append(a)
        y_window.append(data[i+t])
    # conversion to np.ndarray -> then torch.Tensor because of python list of np.ndarray
    return torch.tensor(np.array(X_window)).unsqueeze(-1), torch.tensor(np.array(y_window)).unsqueeze(-1)

#### GRU

In [ ]:
models = {turb_id: GRU(1, 16, 1) for turb_id in data['TurbID'].unique()}

lr = 0.01
epochs = 100
batch_size = 512
turbine_train_losses, turbine_val_losses = {}, {}
for turb_id, model in models.items():
    optimizer = torch.optim.Adam(list(model.parameters()), lr=lr)
    criterion = torch.nn.MSELoss()

    model.to(device)
    criterion.to(device)

    X_train_window, y_train_window = format_sliding_window(train_data[train_data['TurbID'] == turb_id]['P_norm'].to_numpy())
    train_loader = DataLoader(WindFarmDataset(X_train_window, y_train_window), batch_size=batch_size)
    X_val_window, y_val_window = format_sliding_window(val_data[val_data['TurbID'] == turb_id]['P_norm'].to_numpy())
    val_loader = DataLoader(WindFarmDataset(X_val_window, y_val_window), batch_size=batch_size)

    print(f"Training GRU for turbine {turb_id}")
    models[turb_id], train_losses, val_losses = train(model, train_loader, val_loader, optimizer, criterion, epochs=epochs, patience=10)
    print("\n\n")
    turbine_train_losses[turb_id] = train_losses
    turbine_val_losses[turb_id] = val_losses

Training GRU for turbine 1
Epoch: 1;	training loss: 0.0256;	validation loss: 0.0043
Epoch: 10;	training loss: 0.0073;	validation loss: 0.0037
Epoch: 20;	training loss: 0.0072;	validation loss: 0.0037
Stopping early on epoch 27



Training GRU for turbine 2
Epoch: 1;	training loss: 0.0625;	validation loss: 0.0052
Epoch: 10;	training loss: 0.0066;	validation loss: 0.0037
Epoch: 20;	training loss: 0.0066;	validation loss: 0.0036
Epoch: 30;	training loss: 0.0065;	validation loss: 0.0035
Epoch: 40;	training loss: 0.0065;	validation loss: 0.0035
Epoch: 50;	training loss: 0.0066;	validation loss: 0.0035
Epoch: 60;	training loss: 0.0065;	validation loss: 0.0034
Epoch: 70;	training loss: 0.0065;	validation loss: 0.0034
Stopping early on epoch 72



Training GRU for turbine 3
Epoch: 1;	training loss: 0.0350;	validation loss: 0.0047
